In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import lightgbm as lgb
import seaborn as sns
from pprint import pprint
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import shuffle

In [ ]:
#Four Differrent Ensemble Algorithms
#1. Bagging - Random Forest
#2. Boosting - Adapative Boosting, traditional Gradient Boosting (abondoned), Light GBM(better version of GBT), Extreme Gradient Boosting (Better version of Gradient Boosting)
#3. Stacking - StackingCVRegressor(first layer - lgbm/xgb, svr, knn, meta - lasso/ridge)
#4. Voting - Weighted Averaging, mean, median, VotingRegressor

In [ ]:
#G1 = C1, N2
#G2 = C2, C3, C4, H2S
#G3 = C6+

In [ ]:
#data import and re-organize
df = pd.read_csv('C:/Users/cwu/OneDrive - EERC/Desktop/CO2 MMP ML/MMP_Impure_G1.csv')
df.head()
df2=np.array(df)

Shuffled_data = shuffle(df, random_state = 25).reset_index().drop('index', axis=1) 

y_shuffled= Shuffled_data['MMP']
x_shuffled=Shuffled_data.drop('MMP', axis=1)

In [ ]:
#train set and test set split
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler().fit(x_shuffled)

#skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=7)
skf = ShuffleSplit(n_splits=5,test_size=0.25,random_state=7)
for train_index,test_index in skf.split(x_shuffled,y_shuffled):
    trainx1, testx1 = x_shuffled.iloc[train_index],x_shuffled.iloc[test_index]
    trainy, testy = y_shuffled.iloc[train_index],y_shuffled.iloc[test_index]

trainx = scaler.fit_transform(trainx1)
testx = scaler.transform(testx1)

In [ ]:
#Random Forest
# Grid Search
random_seed=44
random_forest_seed=np.random.randint(low=1,high=230)

random_forest_hp_range={'n_estimators':[100,200,400,800,1600,1650,2000],
                          'max_features':[10,11,12,13,14,15],
                          'max_depth':[10,100,200,280,300,400],
                          'min_samples_split':[2,3], # Greater than 1
                          'min_samples_leaf':[1,2]
                          }
random_forest_model_test_base=RandomForestRegressor()
random_forest_model_test_random=GridSearchCV(estimator=random_forest_model_test_base,
                                               param_grid=random_forest_hp_range,
                                               cv=5,
                                               verbose=1,
                                               n_jobs=-1,
                                               scoring="neg_mean_squared_error")
grid_result_RF = random_forest_model_test_random.fit(trainx,trainy)

# summarize results
print("Best: %f using %s" % (grid_result_RF.best_score_, grid_result_RF.best_params_))

means = grid_result_RF.cv_results_['mean_test_score']
stds = grid_result_RF.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_RF.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
#Random Forest
# Predict test set data

best_model_RF = grid_result_RF.best_estimator_
best_model_RF.fit(trainx,trainy)
random_forest_predict=best_model_RF.predict(testx)
random_forest_error=random_forest_predict-testy
random_forest_relative_error=sum(abs(random_forest_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,random_forest_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("Random Forest")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(random_forest_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("Random Forest")
plt.grid(False)

# Verify the accuracy

random_forest_pearson_r=stats.pearsonr(testy,random_forest_predict)
random_forest_R2=metrics.r2_score(testy,random_forest_predict)
random_forest_RMSE=metrics.mean_squared_error(testy,random_forest_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(random_forest_pearson_r[0],
                                                                        random_forest_RMSE,random_forest_R2))
print('Relative Error is', random_forest_relative_error)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#ADABoost

param_ADA = {"estimator__max_depth": [1,3,5,7,9],
            "estimator__min_samples_leaf":[1,2,4,8,10],
            "estimator__max_leaf_nodes":[5,10,20,40,80],
            'n_estimators':[20,40,100,200],
            'learning_rate':[0.001, 0.01, 0.1, 1.0]}       

DTC = DecisionTreeRegressor()
ADA_DT = AdaBoostRegressor(estimator=DTC,learning_rate=1,n_estimators=50,random_state=7)
                           
gsearch_ADADT = GridSearchCV(estimator = ADA_DT,param_grid = param_ADA,cv=5,scoring="neg_mean_squared_error",n_jobs=-1)
grid_result_ADADT = gsearch_ADADT.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_ADADT.best_score_, grid_result_ADADT.best_params_))

means = grid_result_ADADT.cv_results_['mean_test_score']
stds = grid_result_ADADT.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_ADADT.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
# Predict test set data

best_model_ADADT = grid_result_ADADT.best_estimator_
best_model_ADADT.fit(trainx,trainy)
ADADT_predict=best_model_ADADT.predict(testx)
ADADT_error=ADADT_predict-testy
ADADT_relative_error=sum(abs(ADADT_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,ADADT_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("AdaBoost")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(ADADT_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("AdaBoost")
plt.grid(False)

# Verify the accuracy

ADADT_pearson_r=stats.pearsonr(testy,ADADT_predict)
ADADT_R2=metrics.r2_score(testy,ADADT_predict)
ADADT_RMSE=metrics.mean_squared_error(testy,ADADT_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(ADADT_pearson_r[0],
                                                                        ADADT_RMSE,ADADT_R2))
print('Relative Error is', ADADT_relative_error) 

In [ ]:
#LGBM

parameters = {
    'num_iterations': [100,500,1000],
    'learning_rate':[0.01, 0.1, 1.0],
    'num_leaves':[15,31,70],    
    'max_depth' :[5,7,15],
    'min_child_samples':[15,20,25],
    'colsample_bytree': [0.6,0.8],
    'subsample': [0.6,0.8],
    'subsample_freq': [100,200,400]
            }

grid_LGBM = GridSearchCV(lgb.LGBMRegressor(), parameters, cv=5, n_jobs=-1,scoring="neg_mean_squared_error")
grid_result_LGBM = grid_LGBM.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_LGBM.best_score_, grid_result_LGBM.best_params_))

means = grid_result_LGBM.cv_results_['mean_test_score']
stds = grid_result_LGBM.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_LGBM.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
# Predict test set data

best_model_LGBM = grid_result_LGBM.best_estimator_
best_model_LGBM.fit(trainx,trainy)
LGBM_predict=best_model_LGBM.predict(testx)
LGBM_error=LGBM_predict-testy
LGBM_relative_error=sum(abs(LGBM_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,LGBM_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("LGBM")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(LGBM_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("LGBM")
plt.grid(False)

# Verify the accuracy

LGBM_pearson_r=stats.pearsonr(testy,LGBM_predict)
LGBM_R2=metrics.r2_score(testy,LGBM_predict)
LGBM_RMSE=metrics.mean_squared_error(testy,LGBM_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(LGBM_pearson_r[0],
                                                                        LGBM_RMSE,LGBM_R2))
print('Relative Error is', LGBM_relative_error)                                                                        

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

parameters_XGB = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'learning_rate': [0.2,0.3,0.5,0.7], #so called `eta` value
              'max_depth': [2,4,6],
              'min_child_weight': [4,5],
              'subsample': [i/10.0 for i in range(6,11)],
              'colsample_bytree': [i/10.0 for i in range(6,11)],
              'n_estimators': [50,100,250,500]}

grid_XGB = GridSearchCV(XGBRegressor(), parameters_XGB, cv=5, n_jobs=-1,scoring="neg_mean_squared_error")
grid_result_XGB = grid_XGB.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_XGB.best_score_, grid_result_XGB.best_params_))

means = grid_result_XGB.cv_results_['mean_test_score']
stds = grid_result_XGB.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_XGB.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
# Predict test set data

best_model_XGB = grid_result_XGB.best_estimator_
best_model_XGB.fit(trainx,trainy)
XGB_predict=best_model_XGB.predict(testx)
XGB_error=XGB_predict-testy
XGB_relative_error=sum(abs(XGB_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,XGB_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("XGBoost")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(XGB_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("XGBoost")
plt.grid(False)

# Verify the accuracy

XGB_pearson_r=stats.pearsonr(testy,XGB_predict)
XGB_R2=metrics.r2_score(testy,XGB_predict)
XGB_RMSE=metrics.mean_squared_error(testy,XGB_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(XGB_pearson_r[0],
                                                                        XGB_RMSE,XGB_R2))
print('Relative Error is', XGB_relative_error)                                                                           

In [ ]:
from sklearn.svm import SVR

parameters_SVR = {'kernel':['poly','rbf','sigmoid'], 
              'degree': [3,8], 
              'gamma': [1,0.1,0.01,0.001,0.0001],
              'C': [0.1,1,10,100,1000]
              }

grid_SVR = GridSearchCV(SVR(), parameters_SVR, cv=5, n_jobs=-1,scoring="neg_mean_squared_error")
grid_result_SVR = grid_SVR.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_SVR.best_score_, grid_result_SVR.best_params_))

means = grid_result_SVR.cv_results_['mean_test_score']
stds = grid_result_SVR.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_SVR.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

parameters_KNN = {'n_neighbors':[i for i in range(1,11)], 
              'weights': ['uniform','distance'], 
              'p': [1,2]
              }

grid_KNN = GridSearchCV(KNeighborsRegressor(), parameters_KNN, cv=5, n_jobs=-1,scoring="neg_mean_squared_error")
grid_result_KNN = grid_KNN.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_KNN.best_score_, grid_result_KNN.best_params_))

means = grid_result_KNN.cv_results_['mean_test_score']
stds = grid_result_KNN.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_KNN.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso

#meta = ridge, stacking

best_model_SVR = grid_result_SVR.best_estimator_
best_model_KNN = grid_result_KNN.best_estimator_

xgb_best = best_model_XGB
svr_best = best_model_SVR
knn_best = best_model_KNN
ridge = Ridge()
lasso = Lasso()

stack = StackingCVRegressor(regressors=[xgb_best,svr_best,knn_best],
                            meta_regressor = ridge,
                            random_state = 7,
                            use_features_in_secondary = True
                            )

#parameters_Stack = {'meta_regressor__penalty':['l1','l2'], 
              #'meta_regressor__C': [0.1,1,10,100] 
              #}

parameters_Stack = {
                    'meta_regressor':[ridge,lasso],
                    'meta_regressor__alpha':[0.1,1,10]
                    }

grid_Stack = GridSearchCV(estimator=stack, 
                            param_grid = parameters_Stack,
                            cv=5, 
                            n_jobs=-1,
                            scoring="neg_mean_squared_error")
grid_result_Stack = grid_Stack.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_Stack.best_score_, grid_result_Stack.best_params_))

means = grid_result_Stack.cv_results_['mean_test_score']
stds = grid_result_Stack.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_Stack.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))


In [ ]:
# Predict test set data

best_model_Stack = grid_result_Stack.best_estimator_
best_model_Stack.fit(trainx,trainy)
Stack_predict=best_model_Stack.predict(testx)
Stack_error=Stack_predict-testy
Stack_relative_error=sum(abs(Stack_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,Stack_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("Stack_ridge")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(Stack_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("Stack_ridge")
plt.grid(False)

# Verify the accuracy

Stack_pearson_r=stats.pearsonr(testy,Stack_predict)
Stack_R2=metrics.r2_score(testy,Stack_predict)
Stack_RMSE=metrics.mean_squared_error(testy,Stack_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(Stack_pearson_r[0],
                                                                        Stack_RMSE,Stack_R2))
print('Relative Error is', Stack_relative_error)  

In [ ]:
from sklearn import metrics
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras import preprocessing
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.constraints import max_norm
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [ ]:
def create_model(learn_rate =0.01,neurons=12,dropoutrate=0.1):
    neurons_2 = neurons/2
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=15,kernel_initializer='normal', activation='relu', kernel_constraint = max_norm(3)))
    model.add(Dropout(dropoutrate))
    model.add(Dense(neurons_2,kernel_initializer='normal', activation='relu', kernel_constraint=max_norm(3)))
    model.add(Dropout(dropoutrate))
    model.add(Dense(1,kernel_initializer='normal'))
    
    # Compile model
    optimizer = Adam(learning_rate = learn_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy']) 
    return model

seed = 7
np.random.seed(seed)

#create model
model_1 = KerasRegressor(model=create_model,learn_rate=0.01,neurons=12,dropoutrate=0.1,verbose=0)

#define the grid search parameters (epochs,batch size, optimizer)
neurons = [12,16,32]
batch_size = [20, 40, 80, 100]
epochs = [50,100,150]
learn_rate = [0.001,0.01,0.1]
dropoutrate = [0.1,0.2,0.3,0.4,0.5]
param_1 = dict(batch_size=batch_size, epochs=epochs,learn_rate = learn_rate,neurons=neurons,dropoutrate=dropoutrate)
grid_1 = GridSearchCV(estimator=model_1,param_grid=param_1,cv=5,scoring="neg_mean_squared_error",n_jobs=-1)
grid_result_1 = grid_1.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_1.best_score_, grid_result_1.best_params_))

means = grid_result_1.cv_results_['mean_test_score']
stds = grid_result_1.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_1.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))

In [ ]:
# Predict test set data

best_model_ANN = grid_result_1.best_estimator_
best_model_ANN.fit(trainx,trainy)
ANN_predict=best_model_RF.predict(testx)
ANN_error=ANN_predict-testy
ANN_relative_error=sum(abs(ANN_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,ANN_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("ANN")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(ANN_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("ANN")
plt.grid(False)

# Verify the accuracy

ANN_pearson_r=stats.pearsonr(testy,ANN_predict)
ANN_R2=metrics.r2_score(testy,ANN_predict)
ANN_RMSE=metrics.mean_squared_error(testy,ANN_predict)**0.5
print('Pearson correlation coefficient is {0},, RMSE is {1}, and r2 score is {2}.'.format(ANN_pearson_r[0],
                                                                        ANN_RMSE,ANN_R2))
print('Relative Error is', ANN_relative_error)

In [ ]:
#Employ Softmax to calculated weight factors

import math

z = [random_forest_R2, ADADT_R2, LGBM_R2, XGB_R2, Stack_R2, ANN_R2]
z_exp = [math.exp(i) for i in z]  
print(z_exp) 
sum_z_exp = sum(z_exp)  
print(sum_z_exp)  
softmax = [round(i / sum_z_exp, 3) for i in z_exp]
print(softmax)  

In [ ]:
from sklearn.ensemble import VotingRegressor

r1 = best_model_RF
r2 = best_model_ADADT
r3 = best_model_LGBM
r4 = best_model_XGB
r5 = best_model_Stack
r6 = best_model_ANN

er = VotingRegressor([('RF', r1),('ADA', r2), ('LGBM', r3),('XGB',r4),('Stack',r5),('ANN',r6)],weights=[1,1,1,1,1,1])

parameters_er = {'weights':[(1,1,1,1,1,1),(softmax[0],softmax[1],softmax[2],softmax[3],softmax[4],softmax[5])]}

grid_er = GridSearchCV(estimator=er, 
                        param_grid = parameters_er,
                        cv=5, 
                        n_jobs=-1,
                        scoring="neg_mean_squared_error")
grid_result_er = grid_er.fit(trainx, trainy)

# summarize results
print("Best: %f using %s" % (grid_result_er.best_score_, grid_result_er.best_params_))

means = grid_result_er.cv_results_['mean_test_score']
stds = grid_result_er.cv_results_['std_test_score']
for mean,score,params in zip(means,stds,grid_result_er.cv_results_['params']):
    print("%f (%f) with: %r" % (mean, score, params))


In [ ]:
# Predict test set data

best_model_er = grid_result_er.best_estimator_
best_model_er.fit(trainx,trainy)
er_predict=best_model_er.predict(testx)
er_error=er_predict-testy
er_relative_error=sum(abs(er_predict-testy)/testy)*100/len(testx)

# Draw test plot

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(testy,er_predict)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title("ER")
Lims=[0,100]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

plt.figure(2)
plt.clf()
plt.hist(er_error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title("ER")
plt.grid(False)

# Verify the accuracy

er_pearson_r=stats.pearsonr(testy,er_predict)
er_R2=metrics.r2_score(testy,er_predict)
er_RMSE=metrics.mean_squared_error(testy,er_predict)**0.5
print('Pearson correlation coefficient is {0}, RMSE is {1}, and r2 score is {2}.'.format(er_pearson_r[0],
                                                                        er_RMSE,er_R2))
print('Relative Error is', er_relative_error) 